Notes for the near-term "Can we match some secondary service request to a service request in the previous harvests?"

Maybe. Likely. We'll see.

ogc should be straightforward.
ignore thredds for now.
oai-pmh also okay.
not actually sure about opensearch.

the preprocessing:
    put together the set of base urls as a rough cut (if bases don't match, bail)
    for base urls that match, check the expected query parameters
    if match, relate as parent service.
    if not, bail. 

In [22]:
import os
import csv
import json
import urllib, urlparse
from itertools import izip

In [ ]:
# with open('harvest_baseurls.txt', 'r') as f:
#     base_urls = [a.strip() for a in f.readlines()]

In [10]:
with open('harvest_urls_by_protocol.txt', 'r') as f:
    response_urls = [a.replace('"', '').strip().split('||') for a in f.readlines()[1:]]

In [11]:
print response_urls[:3]

[['http://107.20.228.18/arcgis/services/Wetlands/MapServer/WMSServer?request=GetCapabilities&service=WMS', 'OGC:WMS'], ['http://10ncee.org/?format=feed&type=atom', 'ATOM'], ['http://10ncee.org/?format=feed&type=rss', 'ATOM']]


In [69]:
nesting = {}
for source_url, protocol in response_urls:
    if protocol in ['DDI', 'FGDC', 'WordPress'] \
        or 'demo.mapserver.org' in source_url or 'opendap' in source_url \
        or 'ISO-' in protocol or 'KeywordSearch' in source_url or 'ToolsUI.jnlp?' in source_url:
        continue
    source_url = urllib.unquote(source_url)
    parts = urlparse.urlparse(source_url)
    
    base = urlparse.urlunparse((
        parts.scheme, parts.netloc, parts.path, None, None, None
        ))
    
    bases = nesting.get(base, [])
    params = dict( (k.lower(), v if len(v)>1 else list(set(v))[0] ) 
           for k, v in urlparse.parse_qs(parts.query).iteritems() )
    
    bases.append((protocol, source_url, params))
    nesting[base] = bases

# print [nesting[k] for k in nesting.keys()[:10]]

In [70]:
potentials = {k:v for k, v in nesting.iteritems() if len(v) > 1}

In [71]:
[nesting[k] for k in potentials.keys()[:10]]

[[('ATOM',
   'http://www.openaccessweek.org/profiles/friend/feed?user=0ehknuk3zjb7w&xn_auth=no',
   {'user': '0ehknuk3zjb7w', 'xn_auth': 'no'}),
  ('ATOM',
   'http://www.openaccessweek.org/profiles/friend/feed?user=2c52kvye8oxrd&xn_auth=no',
   {'user': '2c52kvye8oxrd', 'xn_auth': 'no'}),
  ('ATOM',
   'http://www.openaccessweek.org/profiles/friend/feed?user=2mvlhwd1uivbe&xn_auth=no',
   {'user': '2mvlhwd1uivbe', 'xn_auth': 'no'}),
  ('ATOM',
   'http://www.openaccessweek.org/profiles/friend/feed?xn_auth=no&sort=mostRecent&page=1',
   {'page': '1', 'sort': 'mostRecent', 'xn_auth': 'no'})],
 [('ATOM',
   'http://www.nottingham.ac.uk/Biosciences/business/business.aspx?EventsStandardandMiniCombo_MiniList_SyndicationType=2',
   {'eventsstandardandminicombo_minilist_syndicationtype': '2'}),
  ('ATOM',
   'http://www.nottingham.ac.uk/Biosciences/business/business.aspx?EventsStandardandMiniCombo_StandardList_SyndicationType=2',
   {'eventsstandardandminicombo_standardlist_syndicationtype': 

In [74]:
ogc_set = {'service', 'request', 'version'}
oai_set = {'verb', 'metadataprefix'}

def match_dict(in_dict, tuples):
    matches = []
    for protocol, response, params in tuples:
        try:
            if len([k for k, v in params.iteritems() 
                         if [i for i, j in in_dict.iteritems() 
                             if i==k and j.lower()==v.lower() and v and j]]) > 0:
                matches.append(response)
        except:
            print response
            print in_dict, params
    
    return matches
    
def match_keys(in_keys, tuples):
    return [t[1] for t in tuples if set(in_keys).intersection(set(t[2].keys()))]
    

parentage = []
for base_url, response_tuples in potentials.iteritems():
    # so. go through the dict to match
    # certain param sets in the lists.
    
    for protocol, response, params in response_tuples:
        matches = []
        if len(list(ogc_set.intersection(set(params.keys())))) > 1:
            # try for ogc but must have at least two of the three
            if params.get('request', '').lower() != 'getcapabilities':
                if not params.get('service', ''):
                    print 'wtf', params
                to_match = {
                    'service': params.get('service', ''),
                    'version': params.get('version', ''),
                    'request': 'getcapabilities'
                }

                matches = match_dict(to_match, response_tuples)
        elif oai_set.intersection(set(params.keys())):
            if params.get('verb', '') != 'identify':
                to_match = {'verb': 'identify'}
                matches = match_dict(to_match, response_tuples)
        else:
            matches = match_keys(params.keys(), response_tuples)
            
        if matches:
            matches = [m.replace('|', ';;') for m in matches if m != response]
            parentage += izip([response.replace('|', ';;')]*len(matches), matches)
    
parentage[:25]
    

wtf {'layers': 'bluemarble,TRMM_3B42_V6_DAILY', 'format': 'PNG', 'request': 'GetMap', 'height': '800', 'width': '1500', 'version': '1.1.1', 'bbox': '-180,-90,180,90', 'time': '2006-09-11', 'srs': 'epsg:4326'}


[('http://www.openaccessweek.org/profiles/friend/feed?user=0ehknuk3zjb7w&xn_auth=no',
  'http://www.openaccessweek.org/profiles/friend/feed?user=2c52kvye8oxrd&xn_auth=no'),
 ('http://www.openaccessweek.org/profiles/friend/feed?user=0ehknuk3zjb7w&xn_auth=no',
  'http://www.openaccessweek.org/profiles/friend/feed?user=2mvlhwd1uivbe&xn_auth=no'),
 ('http://www.openaccessweek.org/profiles/friend/feed?user=0ehknuk3zjb7w&xn_auth=no',
  'http://www.openaccessweek.org/profiles/friend/feed?xn_auth=no&sort=mostRecent&page=1'),
 ('http://www.openaccessweek.org/profiles/friend/feed?user=2c52kvye8oxrd&xn_auth=no',
  'http://www.openaccessweek.org/profiles/friend/feed?user=0ehknuk3zjb7w&xn_auth=no'),
 ('http://www.openaccessweek.org/profiles/friend/feed?user=2c52kvye8oxrd&xn_auth=no',
  'http://www.openaccessweek.org/profiles/friend/feed?user=2mvlhwd1uivbe&xn_auth=no'),
 ('http://www.openaccessweek.org/profiles/friend/feed?user=2c52kvye8oxrd&xn_auth=no',
  'http://www.openaccessweek.org/profiles/fri

In [75]:
with open('harvest_parentage.csv', 'w') as f:
    f.write('source|possible_match\n')
    f.write('\n'.join(['|'.join(p) for p in parentage]))

In [82]:
ogc_set = {'service', 'request', 'version'}
oai_set = {'verb', 'metadataprefix'}

def match_dict(in_dict, tuples):
    matches = []
    for protocol, response, params in tuples:
        try:
            if len([k for k, v in params.iteritems() 
                         if [i for i, j in in_dict.iteritems() 
                             if i==k and j.lower()==v.lower() and v and j]]) == len(in_dict.keys()):
                matches.append(response)
        except:
            print response
            print in_dict, params
    
    return matches
    
def match_keys(in_keys, tuples):
    return [t[1] for t in tuples if set(in_keys).intersection(set(t[2].keys()))]
    

parentage = []
for base_url, response_tuples in potentials.iteritems():
    # so. go through the dict to match
    # certain param sets in the lists.
    
    for protocol, response, params in response_tuples:
        if protocol not in ['OpenSearch', 'OAI-PMH', 'OGC:WMS',
                            'OGC:WFS', 'OGC:WCS', 'OGC:CSW']:
            continue
        matches = []
        if len(list(ogc_set.intersection(set(params.keys())))) > 1:
            # try for ogc but must have at least two of the three
            if params.get('request', '').lower() != 'getcapabilities':
                if not params.get('service', ''):
                    print 'wtf', params
                to_match = {
                    'service': params.get('service', ''),
                    'version': params.get('version', ''),
                    'request': 'getcapabilities'
                }

                matches = match_dict(to_match, response_tuples)
        elif oai_set.intersection(set(params.keys())):
            if params.get('verb', '') != 'identify':
                to_match = {'verb': 'identify'}
                matches = match_dict(to_match, response_tuples)
        else:
            matches = match_keys(params.keys(), response_tuples)
            
        if matches:
            matches = [m.replace('|', ';;') for m in matches if m != response]
            parentage += izip([protocol]*len(matches), [response.replace('|', ';;')]*len(matches), matches)

In [84]:
with open('harvest_parentage_priority.csv', 'w') as f:
    f.write('protocol|source|possible_match\n')
    f.write('\n'.join(['|'.join(p) for p in parentage]))

### looking for parents in rss/atom responses and checking for existence in the set

In [92]:
from lxml import etree
import os
import json
from itertools import izip

links_to_check = []
counter = 0
total = 0
with open('harvest_urls_feeds.csv', 'r') as f:
    reader = csv.reader(f, delimiter='|')
    for row in reader:
        # get the hash and parse that, looking for links with opensearchdescription types
        # what we're looking for: <link rel='search' type='application/opensearchdescription+xml' 
        #    href='https://www.sciencebase.gov/catalog/opensearchdescription.xml'/>
        filepath = os.path.join('..', 'solr_superset', 
                                'cleaned', row[1].strip() + '_cleaned.json')
        if not os.path.exists(filepath):
            print 'missing file?!: ', filepath
            continue
        
        with open(filepath, 'r') as g:
            data = json.loads(g.read())
        
        content = data.get('content').encode('unicode_escape')
        
        try:
            xml = etree.fromstring(content)
        except:
            print 'failed:', filepath
            continue
        
        # go for a rel with type
        links = xml.xpath('/*/*[local-name()="link" and @rel="search" and @type="application/opensearchdescription+xml"]/@*[local-name()="href"]')
        
        if links:
            counter += 1
            links_to_check += ['|'.join(i) for i in list(izip([row[0].strip()]*len(links), links))]
        
        total += 1

if links_to_check:
    with open('harvest_opensearch_to_check.csv', 'w') as f:
        f.write('source|to_check\n')
        f.write('\n'.join(links_to_check))
    
print '{0} of {1} ({2}%) feed responses contain an OSDD rel link'.format(counter, total, round((float(counter)/total) * 100))    
    

missing file?!:  ../solr_superset/cleaned/raw_content_md5_cleaned.json
failed: ../solr_superset/cleaned/d9356117ed107957e63bd36a0c0637da_cleaned.json
409 of 7532 (5.0%) feed responses contain an OSDD rel link


In [95]:
# and then see if we can match those osdd links with a known link in the harvest.
with open('harvest_opensearch_osdd.txt', 'r') as f:
    opensearches = [a.strip() for a in f.readlines()]

founds = []
for link_to_check in links_to_check:
    parts = link_to_check.strip().split('|')
    source = parts[0]
    link = parts[1]
    if link in opensearches:
        founds.append('|'.join([source, link]))

len(founds)

59

In [96]:
with open('harvest_opensearch_found_parents.csv', 'w') as f:
    f.write('source|link\n')
    f.write('\n'.join(founds))

In [41]:
# what the heck, mapserver demo.

u = 'http://demo.mapserver.org/cgi-bin/wms?SERVICE=WMS&VERSION=1.1.1&REQUEST=GetMap&LAYERS=cities&BBOX=-84.7978536015,41.440374,-75.737539764,45.97524&http:/demo.mapserver.org/cgi-bin/wms?SERVICE=WMS&VERSION=1.1.1&REQUEST=GetMap&LAYERS=cities&BBOX=-84.7978536015,41.440374,-75.737539764,45.97524&FORMAT=png&WIDTH=400&HEIGHT=300&SRS=EPSG:4326&STYLES=&TRANSPARENT=TRUE&SLD=http:/demo.mapserver.org/ogc-demos/map/sld/sld_symbol_external.xml'

parts = urlparse.urlparse(u)
qs = parts.query

params = urlparse.parse_qs(qs)

params

{'BBOX': ['-84.7978536015,41.440374,-75.737539764,45.97524',
  '-84.7978536015,41.440374,-75.737539764,45.97524'],
 'FORMAT': ['png'],
 'HEIGHT': ['300'],
 'LAYERS': ['cities', 'cities'],
 'REQUEST': ['GetMap', 'GetMap'],
 'SERVICE': ['WMS'],
 'SLD': ['http:/demo.mapserver.org/ogc-demos/map/sld/sld_symbol_external.xml'],
 'SRS': ['EPSG:4326'],
 'TRANSPARENT': ['TRUE'],
 'VERSION': ['1.1.1', '1.1.1'],
 'WIDTH': ['400'],
 'http:/demo.mapserver.org/cgi-bin/wms?SERVICE': ['WMS']}

In [52]:
u = 'http://data.opencolorado.org/dataset/history/2012-avondale-tapestry-segmetation-profile?days=7&format=atom'
parts = urlparse.urlparse(u)

urlparse.urlunparse((
    parts.scheme, parts.netloc, parts.path, None, None, None
    ))

'http://data.opencolorado.org/dataset/history/2012-avondale-tapestry-segmetation-profile'